## 问题定义

本问题为求解2D波导空腔中的电磁波传播。

其几何如下所示：

![problem](./resource/problem.png)

整个区域为$[0, 2] \times [0, 2]$的矩形。空间真空，相对介电常数为1.左侧为波导端口，右侧为吸收边界（absorbing boundary, ABC），顶部和底部为理想导电体（Perfect electronic conductor, PEC）。

对于上述定义有如下的PDE和定解条件：

![problem](./resource/equation.png)


### 求解目标

要求求解上述求解区域中的$E_z(x,y)$


## 求解

In [1]:
import os
import warnings

# optional
# set appropriate GPU in case of multi-GPU machine
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="5"

In [2]:
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra.utils import compose

In [3]:
from sympy import Symbol, pi, sin, Number, Eq
from sympy.logic.boolalg import Or

import modulus.sym
from modulus.sym.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.sym.utils.io import csv_to_dict
from modulus.sym.solver import Solver
from modulus.sym.domain import Domain
from modulus.sym.geometry.primitives_2d import Rectangle
from modulus.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)
from modulus.sym.domain.validator import PointwiseValidator
from modulus.sym.domain.inferencer import PointwiseInferencer
from modulus.sym.utils.io.plotter import ValidatorPlotter, InferencerPlotter
from modulus.sym.key import Key
from modulus.sym.eq.pdes.wave_equation import HelmholtzEquation
from modulus.sym.eq.pdes.navier_stokes import GradNormal

In [4]:
cfg = compose(config_path="conf", config_name="config")
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints
print(to_yaml(cfg))

/usr/local/lib/python3.10/dist-packages/modulus/sym/hydra/utils.py:148: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(


training:
  max_steps: 500000
  grad_agg_freq: 1
  rec_results_freq: 5000
  rec_validation_freq: ${training.rec_results_freq}
  rec_inference_freq: ${training.rec_results_freq}
  rec_monitor_freq: ${training.rec_results_freq}
  rec_constraint_freq: ${training.rec_results_freq}
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  amp: false
  amp_dtype: float16
  ntk:
    use_ntk: false
    save_name: null
    run_freq: 1000
graph:
  func_arch: false
  func_arch_allow_partial_hessian: true
stop_criterion:
  metric: null
  min_delta: null
  patience: 50000
  mode: min
  freq: 1000
  strict: false
profiler:
  profile: false
  start_step: 0
  end_step: 100
  name: nvtx
network_dir: outputs
initialization_network_dir: ''
save_filetypes: vtk,npz
summary_histograms: false
jit: true
jit_use_nvfuser: true
jit_arch_mode: only_activation
jit_autograd_nodes: false
cuda_graphs: true
cuda_graph_warmup: 20
find_unused_parameters: false
broadcast_buffers: false
device: ''
debug: fa

### 定义必要组件

In [5]:
wave_number = 32.0  # wave_number = freq/c

x, y = Symbol("x"), Symbol("y")

#### Geo

In [6]:
height = 2
width = 2

rec = Rectangle((0, 0), (width, height))

#### PDE

In [7]:
hm = HelmholtzEquation(u="u", k=wave_number, dim=2)  # 亥姆霍兹方程
gn = GradNormal(T="u", dim=2, time=False)  # 用于计算ABC

#### Model

In [8]:
# 定义简单的全连接网络
# 输入为空间坐标和
# 输出为E_z(x, y)
wave_net = instantiate_arch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("u")],
    frequencies=(
        "axis,diagonal",
        [i / 2.0 for i in range(int(wave_number) * 2 + 1)],
    ),
    frequencies_params=(
        "axis,diagonal",
        [i / 2.0 for i in range(int(wave_number) * 2 + 1)],
    ),
    cfg=cfg.arch.modified_fourier,
)

#### Node

In [9]:
nodes = (
    hm.make_nodes() + gn.make_nodes() + [wave_net.make_node(name="wave_network")]
)

#### Domain

In [10]:
waveguide_domain = Domain()

#### 入口条件（左边界）

In [11]:
eigenmode = [2]
waveguide_port = Number(0)
for k in eigenmode:
    waveguide_port += sin(k * pi * y / height)  # 传入的波

Waveguide_port = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"u": waveguide_port},
    batch_size=cfg.batch_size.Waveguide_port,
    lambda_weighting={"u": 100.0},
    criteria=Eq(x, 0),
)
waveguide_domain.add_constraint(Waveguide_port, "Waveguide_port")

#### 右侧ABC边界

In [12]:
ABC = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"normal_gradient_u": 0.0},
    batch_size=cfg.batch_size.ABC,
    lambda_weighting={"normal_gradient_u": 10.0},
    criteria=Eq(x, width),
)
waveguide_domain.add_constraint(ABC, "ABC")

#### 上下PEC边界

In [13]:
PEC = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"u": 0.0},
    batch_size=cfg.batch_size.PEC,
    lambda_weighting={"u": 100.0},
    criteria=Or(Eq(y, 0), Eq(y, height)),
)
waveguide_domain.add_constraint(PEC, "PEC")

#### PDE约束

In [14]:
Interior = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"helmholtz": 0.0},
    batch_size=cfg.batch_size.Interior,
    bounds={x: (0, width), y: (0, height)},
    lambda_weighting={
        "helmholtz": 1.0 / wave_number**2,
    },
)
waveguide_domain.add_constraint(Interior, "Interior")

#### 验证器以及其他必要组件

In [15]:
# add validation data
file_path = "./openfoam/2Dwaveguide_32_2.csv"
if os.path.exists(to_absolute_path(file_path)):
    mapping = {"x": "x", "y": "y", "u": "u"}
    validation_var = csv_to_dict(to_absolute_path(file_path), mapping)
    validation_invar_numpy = {
        key: value for key, value in validation_var.items() if key in ["x", "y"]
    }
    validation_outvar_numpy = {
        key: value for key, value in validation_var.items() if key in ["u"]
    }

    csv_validator = PointwiseValidator(
        nodes=nodes,
        invar=validation_invar_numpy,
        true_outvar=validation_outvar_numpy,
        batch_size=2048,
        plotter=ValidatorPlotter(),
    )
    waveguide_domain.add_validator(csv_validator)

    # add inferencer data
    csv_inference = PointwiseInferencer(
        nodes=nodes,
        invar=validation_invar_numpy,
        output_names=["u"],
        plotter=InferencerPlotter(),
        batch_size=2048,
    )
    waveguide_domain.add_inferencer(csv_inference, "inf_data")
else:
    warnings.warn(
        f"Directory {file_path} does not exist. Will skip adding validators. Please download the additional files from NGC https://catalog.ngc.nvidia.com/orgs/nvidia/teams/modulus/resources/modulus_sym_examples_supplemental_materials"
    )

### 求解器以及求解

In [16]:
# 定义求解器
slv = Solver(cfg, waveguide_domain)

手动加载日志系统

In [17]:
import logging
# logging.getLogger().addHandler(logging.StreamHandler())
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

启动求解

In [ ]:
slv.solve()

2024-03-01 08:49:46,051 - WARNING - Installed PyTorch version 2.2.0a0+81ea7a4 is not TorchScript supported in Modulus. Version 2.1.0a0+4136153 is officially supported.
2024-03-01 08:49:46,060 - INFO - attempting to restore from: /workspace/18_2D_WaveguideCavity/outputs
2024-03-01 08:49:46,061 - WARNING - optimizer checkpoint not found
2024-03-01 08:49:46,061 - WARNING - model wave_network.0.pth not found
2024-03-01 08:49:47,731 - INFO - [step:          0] record constraint batch time:  1.515e-01s
2024-03-01 08:49:48,952 - INFO - [step:          0] record validators time:  1.219e+00s
2024-03-01 08:49:49,410 - INFO - [step:          0] record inferencers time:  4.408e-01s
2024-03-01 08:49:49,486 - INFO - [step:          0] saved checkpoint to /workspace/18_2D_WaveguideCavity/outputs
2024-03-01 08:49:49,487 - INFO - [step:          0] loss:  1.825e+06
2024-03-01 08:49:51,982 - INFO - Attempting cuda graph building, this may take a bit...
2024-03-01 08:50:09,586 - INFO - [step:        100]

2024-03-01 08:59:07,395 - INFO - [step:       6400] loss:  5.423e+01, time/iteration:  8.308e+01 ms
2024-03-01 08:59:16,404 - INFO - [step:       6500] loss:  5.058e+01, time/iteration:  9.007e+01 ms
2024-03-01 08:59:25,420 - INFO - [step:       6600] loss:  4.742e+01, time/iteration:  9.014e+01 ms
2024-03-01 08:59:33,890 - INFO - [step:       6700] loss:  4.830e+01, time/iteration:  8.469e+01 ms
2024-03-01 08:59:40,211 - INFO - [step:       6800] loss:  5.056e+01, time/iteration:  6.320e+01 ms
2024-03-01 08:59:46,142 - INFO - [step:       6900] loss:  5.223e+01, time/iteration:  5.930e+01 ms
2024-03-01 08:59:53,422 - INFO - [step:       7000] saved checkpoint to /workspace/18_2D_WaveguideCavity/outputs
2024-03-01 08:59:53,426 - INFO - [step:       7000] loss:  3.906e+01, time/iteration:  7.281e+01 ms
2024-03-01 09:00:02,418 - INFO - [step:       7100] loss:  3.822e+01, time/iteration:  8.991e+01 ms
2024-03-01 09:00:11,422 - INFO - [step:       7200] loss:  4.282e+01, time/iteration:  

2024-03-01 09:09:20,479 - INFO - [step:      13600] loss:  9.326e+00, time/iteration:  9.027e+01 ms
2024-03-01 09:09:28,860 - INFO - [step:      13700] loss:  8.751e+00, time/iteration:  8.380e+01 ms
2024-03-01 09:09:37,745 - INFO - [step:      13800] loss:  9.797e+00, time/iteration:  8.884e+01 ms
2024-03-01 09:09:46,715 - INFO - [step:      13900] loss:  9.539e+00, time/iteration:  8.968e+01 ms
2024-03-01 09:09:56,186 - INFO - [step:      14000] saved checkpoint to /workspace/18_2D_WaveguideCavity/outputs
2024-03-01 09:09:56,188 - INFO - [step:      14000] loss:  8.191e+00, time/iteration:  9.471e+01 ms
2024-03-01 09:10:04,703 - INFO - [step:      14100] loss:  8.487e+00, time/iteration:  8.515e+01 ms
2024-03-01 09:10:13,408 - INFO - [step:      14200] loss:  8.207e+00, time/iteration:  8.703e+01 ms
2024-03-01 09:10:22,305 - INFO - [step:      14300] loss:  7.282e+00, time/iteration:  8.896e+01 ms
2024-03-01 09:10:31,276 - INFO - [step:      14400] loss:  8.202e+00, time/iteration:  

2024-03-01 09:19:06,558 - INFO - [step:      20500] loss:  2.645e+00, time/iteration:  9.080e+01 ms
2024-03-01 09:19:15,381 - INFO - [step:      20600] loss:  2.547e+00, time/iteration:  8.822e+01 ms
2024-03-01 09:19:23,998 - INFO - [step:      20700] loss:  2.394e+00, time/iteration:  8.615e+01 ms
2024-03-01 09:19:32,504 - INFO - [step:      20800] loss:  2.486e+00, time/iteration:  8.504e+01 ms
2024-03-01 09:19:41,268 - INFO - [step:      20900] loss:  2.349e+00, time/iteration:  8.762e+01 ms
2024-03-01 09:19:50,694 - INFO - [step:      21000] saved checkpoint to /workspace/18_2D_WaveguideCavity/outputs
2024-03-01 09:19:50,696 - INFO - [step:      21000] loss:  2.329e+00, time/iteration:  9.426e+01 ms
2024-03-01 09:19:59,475 - INFO - [step:      21100] loss:  2.316e+00, time/iteration:  8.778e+01 ms
2024-03-01 09:20:07,669 - INFO - [step:      21200] loss:  2.231e+00, time/iteration:  8.193e+01 ms
2024-03-01 09:20:16,646 - INFO - [step:      21300] loss:  2.274e+00, time/iteration:  

2024-03-01 09:29:26,244 - INFO - [step:      27700] loss:  1.037e+00, time/iteration:  8.333e+01 ms
2024-03-01 09:29:35,128 - INFO - [step:      27800] loss:  1.010e+00, time/iteration:  8.881e+01 ms
2024-03-01 09:29:43,803 - INFO - [step:      27900] loss:  1.049e+00, time/iteration:  8.673e+01 ms
2024-03-01 09:29:53,000 - INFO - [step:      28000] saved checkpoint to /workspace/18_2D_WaveguideCavity/outputs
2024-03-01 09:29:53,001 - INFO - [step:      28000] loss:  9.246e-01, time/iteration:  9.196e+01 ms
2024-03-01 09:29:59,502 - INFO - [step:      28100] loss:  9.966e-01, time/iteration:  6.500e+01 ms
2024-03-01 09:30:05,223 - INFO - [step:      28200] loss:  9.940e-01, time/iteration:  5.719e+01 ms
2024-03-01 09:30:12,534 - INFO - [step:      28300] loss:  9.884e-01, time/iteration:  7.309e+01 ms
2024-03-01 09:30:21,400 - INFO - [step:      28400] loss:  1.011e+00, time/iteration:  8.864e+01 ms
2024-03-01 09:30:30,304 - INFO - [step:      28500] loss:  1.068e+00, time/iteration:  

...

2024-03-01 13:04:22,122 - INFO - [step:     198500] loss:  9.532e-03, time/iteration:  5.628e+01 ms
2024-03-01 13:04:28,402 - INFO - [step:     198600] loss:  8.607e-03, time/iteration:  6.279e+01 ms
2024-03-01 13:04:34,760 - INFO - [step:     198700] loss:  8.645e-03, time/iteration:  6.356e+01 ms
2024-03-01 13:04:41,219 - INFO - [step:     198800] loss:  5.245e-03, time/iteration:  6.457e+01 ms
2024-03-01 13:04:47,635 - INFO - [step:     198900] loss:  5.828e-03, time/iteration:  6.414e+01 ms
2024-03-01 13:04:54,675 - INFO - [step:     199000] saved checkpoint to /workspace/18_2D_WaveguideCavity/outputs
2024-03-01 13:04:54,679 - INFO - [step:     199000] loss:  5.703e-03, time/iteration:  7.040e+01 ms
2024-03-01 13:05:00,564 - INFO - [step:     199100] loss:  5.597e-03, time/iteration:  5.883e+01 ms
2024-03-01 13:05:06,912 - INFO - [step:     199200] loss:  6.535e-03, time/iteration:  6.347e+01 ms
2024-03-01 13:05:13,229 - INFO - [step:     199300] loss:  5.979e-03, time/iteration:  

2024-03-01 13:11:54,721 - INFO - [step:     205400] loss:  6.248e-03, time/iteration:  6.774e+01 ms
2024-03-01 13:12:01,499 - INFO - [step:     205500] loss:  5.376e-03, time/iteration:  6.776e+01 ms
2024-03-01 13:12:07,930 - INFO - [step:     205600] loss:  4.671e-03, time/iteration:  6.427e+01 ms
2024-03-01 13:12:14,211 - INFO - [step:     205700] loss:  5.725e-03, time/iteration:  6.279e+01 ms
2024-03-01 13:12:20,723 - INFO - [step:     205800] loss:  5.309e-03, time/iteration:  6.510e+01 ms
2024-03-01 13:12:27,343 - INFO - [step:     205900] loss:  5.076e-03, time/iteration:  6.617e+01 ms
2024-03-01 13:12:34,494 - INFO - [step:     206000] saved checkpoint to /workspace/18_2D_WaveguideCavity/outputs
2024-03-01 13:12:34,496 - INFO - [step:     206000] loss:  4.233e-03, time/iteration:  7.151e+01 ms
2024-03-01 13:12:41,138 - INFO - [step:     206100] loss:  5.756e-03, time/iteration:  6.640e+01 ms
2024-03-01 13:12:47,275 - INFO - [step:     206200] loss:  4.824e-03, time/iteration:  

2024-03-01 13:19:42,077 - INFO - [step:     212600] loss:  5.445e-03, time/iteration:  6.590e+01 ms
2024-03-01 13:19:48,491 - INFO - [step:     212700] loss:  3.860e-03, time/iteration:  6.412e+01 ms
2024-03-01 13:19:54,986 - INFO - [step:     212800] loss:  4.943e-03, time/iteration:  6.493e+01 ms
2024-03-01 13:20:01,635 - INFO - [step:     212900] loss:  5.148e-03, time/iteration:  6.647e+01 ms
2024-03-01 13:20:08,245 - INFO - [step:     213000] saved checkpoint to /workspace/18_2D_WaveguideCavity/outputs
2024-03-01 13:20:08,247 - INFO - [step:     213000] loss:  5.697e-03, time/iteration:  6.609e+01 ms
2024-03-01 13:20:14,182 - INFO - [step:     213100] loss:  5.563e-03, time/iteration:  5.935e+01 ms
2024-03-01 13:20:20,473 - INFO - [step:     213200] loss:  4.626e-03, time/iteration:  6.289e+01 ms
2024-03-01 13:20:26,978 - INFO - [step:     213300] loss:  8.608e-03, time/iteration:  6.502e+01 ms
2024-03-01 13:20:33,452 - INFO - [step:     213400] loss:  3.956e-03, time/iteration:  

2024-03-01 13:27:26,882 - INFO - [step:     219800] loss:  4.029e-03, time/iteration:  6.092e+01 ms
2024-03-01 13:27:32,903 - INFO - [step:     219900] loss:  3.156e-03, time/iteration:  6.020e+01 ms
2024-03-01 13:27:39,593 - INFO - [step:     220000] record constraint batch time:  1.430e-01s
2024-03-01 13:27:40,779 - INFO - [step:     220000] record validators time:  1.184e+00s
2024-03-01 13:27:41,325 - INFO - [step:     220000] record inferencers time:  5.230e-01s
2024-03-01 13:27:41,404 - INFO - [step:     220000] saved checkpoint to /workspace/18_2D_WaveguideCavity/outputs
2024-03-01 13:27:41,406 - INFO - [step:     220000] loss:  4.928e-03, time/iteration:  8.498e+01 ms
2024-03-01 13:27:48,008 - INFO - [step:     220100] loss:  3.393e-03, time/iteration:  6.600e+01 ms
2024-03-01 13:27:54,151 - INFO - [step:     220200] loss:  3.550e-03, time/iteration:  6.141e+01 ms
2024-03-01 13:28:00,328 - INFO - [step:     220300] loss:  5.216e-03, time/iteration:  6.173e+01 ms
2024-03-01 13:28

2024-03-01 13:34:29,123 - INFO - [step:     226700] loss:  3.704e-03, time/iteration:  6.154e+01 ms
2024-03-01 13:34:35,031 - INFO - [step:     226800] loss:  3.607e-03, time/iteration:  5.906e+01 ms
2024-03-01 13:34:40,993 - INFO - [step:     226900] loss:  5.284e-03, time/iteration:  5.958e+01 ms
2024-03-01 13:34:47,630 - INFO - [step:     227000] saved checkpoint to /workspace/18_2D_WaveguideCavity/outputs
2024-03-01 13:34:47,632 - INFO - [step:     227000] loss:  3.320e-03, time/iteration:  6.637e+01 ms
2024-03-01 13:34:53,521 - INFO - [step:     227100] loss:  3.305e-03, time/iteration:  5.888e+01 ms
2024-03-01 13:34:59,423 - INFO - [step:     227200] loss:  4.864e-03, time/iteration:  5.899e+01 ms
2024-03-01 13:35:05,203 - INFO - [step:     227300] loss:  3.468e-03, time/iteration:  5.778e+01 ms
2024-03-01 13:35:11,056 - INFO - [step:     227400] loss:  4.112e-03, time/iteration:  5.850e+01 ms
2024-03-01 13:35:17,064 - INFO - [step:     227500] loss:  3.006e-03, time/iteration:  

### 后处理以及可视化

对于jupyter，比较方便的方法是使用matplotlib

此外，还可以使用tensorboard以及Paraview

![u](./outputs/validators/validator_u.png)